In [55]:
import requests
import requests_cache
requests_cache.install_cache('cache')
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
import json
from pprint import pprint
API_KEY = 'pk.eyJ1IjoibnlvdTA0NSIsImEiOiJjaXJ6bDYxeWcwMXRzMnlwa2tmbzduNmd0In0.gLrTUpqhS3PYYy_AtCp4Gw'
BASE_URL = f"https://api.mapbox.com/search/searchbox/v1/category/services?access_token={API_KEY}&language=en&limit=25"

In [9]:
sa2 = gpd.read_file("statsnzstatistical-area-2-2022-clipped-generalised-SHP.zip")
sa2

,SA22022_V1,SA22022__1,SA22022__2,LAND_AREA_,AREA_SQ_KM,Shape_Leng,geometry
0,100100,North Cape,North Cape,829.225537,829.225537,415887.784243,"MULTIPOLYGON (((173.15304 -34.75709, 173.15302..."
1,100200,Rangaunu Harbour,Rangaunu Harbour,273.987277,273.987277,149055.116424,"MULTIPOLYGON (((173.26482 -34.99699, 173.26501..."
2,100400,Karikari Peninsula,Karikari Peninsula,174.440751,174.440751,140149.099238,"MULTIPOLYGON (((173.30152 -35.00198, 173.30124..."
3,100500,Tangonge,Tangonge,177.176939,177.176939,101448.150296,"POLYGON ((173.24283 -35.06012, 173.25488 -35.0..."
4,100900,Rangitihi,Rangitihi,84.553189,84.553189,60616.141491,"POLYGON ((173.29023 -35.06998, 173.29028 -35.0..."
...,...,...,...,...,...,...,...
2173,362300,Kennington-Tisbury,Kennington-Tisbury,36.983391,36.983391,38816.533421,"POLYGON ((168.45877 -46.38944, 168.45878 -46.3..."
2174,362400,Newfield,Newfield,2.005553,2.005553,6488.270133,"POLYGON ((168.39621 -46.41171, 168.39838 -46.4..."
2175,362500,Strathern,Strathern,0.983959,0.983959,4203.835163,"POLYGON ((168.36193 -46.43233, 168.36146 -46.4..."
2176,362600,Elizabeth Park,Elizabeth Park,1.030253,1.030253,5085.466054,"POLYGON ((168.39105 -46.42704, 168.39103 -46.4..."


In [41]:
sa2["retrieved_at"] = pd.NA
sa2["n_POIs"] = pd.NA
sa2.drop(columns="geometry").to_csv("locations.csv", index=False)

In [48]:
sample = sa2.sample(10)
sample

,SA22022_V1,SA22022__1,SA22022__2,LAND_AREA_,AREA_SQ_KM,Shape_Leng,geometry,retrieved_at,n_POIs
1556,237200,Waikanae Park,Waikanae Park,7.359810,7.359810,14546.699554,"POLYGON ((175.05324 -40.84423, 175.06394 -40.8...",<NA>,<NA>
620,358400,Otautau,Otautau,703.249413,703.249413,155041.807477,"POLYGON ((167.88981 -46.03714, 167.89209 -46.0...",<NA>,<NA>
1592,240700,Cannons Creek North,Cannons Creek North,1.057064,1.057064,6103.907623,"POLYGON ((174.86286 -41.13252, 174.86140 -41.1...",<NA>,<NA>
950,150100,Ōtāhuhu Central,Otahuhu Central,1.271227,1.271227,7848.083995,"POLYGON ((174.83900 -36.93816, 174.83932 -36.9...",<NA>,<NA>
1887,320700,Ilam South,Ilam South,0.911775,0.911775,5496.516868,"POLYGON ((172.57345 -43.51693, 172.57515 -43.5...",<NA>,<NA>
484,333100,Banks Peninsula South,Banks Peninsula South,371.610981,377.862913,181238.836589,"POLYGON ((172.66229 -43.68773, 172.67628 -43.6...",<NA>,<NA>
1110,173100,Eureka-Tauwhare,Eureka-Tauwhare,135.569365,135.569365,65989.866069,"POLYGON ((175.43812 -37.69567, 175.43838 -37.6...",<NA>,<NA>
1032,162000,Takanini West,Takanini West,0.686327,0.686327,4577.055752,"POLYGON ((174.92526 -37.04240, 174.92529 -37.0...",<NA>,<NA>
1659,249000,Broadmeadows,Broadmeadows,0.642137,0.642137,3753.041367,"POLYGON ((174.79257 -41.22976, 174.79309 -41.2...",<NA>,<NA>
956,158100,Ormiston South,Ormiston South,1.703467,1.703467,5866.134260,"POLYGON ((174.92198 -36.96751, 174.92271 -36.9...",<NA>,<NA>


In [56]:
results = {}
for i, b in tqdm(sample.bounds.iterrows(), total=len(sample)):
    bbox = f"{b.minx},{b.miny},{b.maxx},{b.maxy}"
    geojson = requests.get(BASE_URL, params={"bbox": bbox}).json()
    print(sample.loc[i])
    pprint(geojson)
    for feature in geojson["features"]:
        results[feature["properties"]["mapbox_id"]] = feature
    sample.retrieved_at[i] = pd.Timestamp.now()
    sample.n_POIs[i] = len(geojson["features"])
sample

  0%|          | 0/10 [00:00<?, ?it/s]

SA22022_V1                                                 237200
SA22022__1                                          Waikanae Park
SA22022__2                                          Waikanae Park
LAND_AREA_                                                7.35981
AREA_SQ_KM                                                7.35981
Shape_Leng                                           14546.699554
geometry        POLYGON ((175.05323626656227 -40.8442252329052...
retrieved_at                           2023-08-22 17:15:44.728922
n_POIs                                                          0
Name: 1556, dtype: object
{'attribution': '© 2023 Mapbox and its suppliers. All rights reserved. Use of '
                'this data is subject to the Mapbox Terms of Service. '
                '(https://www.mapbox.com/about/maps/)',
 'features': [],
 'type': 'FeatureCollection'}
SA22022_V1                                                 358400
SA22022__1                                                O

<ipython-input-56-dc069f375895>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.retrieved_at[i] = pd.Timestamp.now()
<ipython-input-56-dc069f375895>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.n_POIs[i] = len(geojson["features"])


'name': 'Cheap Towing',
                              'place_formatted': 'Auckland 1062, New Zealand',
                              'poi_category': ['mechanic', 'services'],
                              'poi_category_ids': ['auto_repair', 'services']},
               'type': 'Feature'},
              {'geometry': {'coordinates': [174.836594, -36.94354],
                            'type': 'Point'},
               'properties': {'address': '68 Huia Road',
                              'context': {'address': {'address_number': '68',
                                                      'name': '68 Huia Road',
                                                      'street_name': 'Huia '
                                                                     'Road'},
                                          'country': {'country_code': 'NZ',
                                                      'country_code_alpha_3': 'NZL',
                                                      'name': 'New 

,SA22022_V1,SA22022__1,SA22022__2,LAND_AREA_,AREA_SQ_KM,Shape_Leng,geometry,retrieved_at,n_POIs
1556,237200,Waikanae Park,Waikanae Park,7.359810,7.359810,14546.699554,"POLYGON ((175.05324 -40.84423, 175.06394 -40.8...",2023-08-22 17:19:39.913543,0
620,358400,Otautau,Otautau,703.249413,703.249413,155041.807477,"POLYGON ((167.88981 -46.03714, 167.89209 -46.0...",2023-08-22 17:19:39.919443,0
1592,240700,Cannons Creek North,Cannons Creek North,1.057064,1.057064,6103.907623,"POLYGON ((174.86286 -41.13252, 174.86140 -41.1...",2023-08-22 17:19:39.923524,0
950,150100,Ōtāhuhu Central,Otahuhu Central,1.271227,1.271227,7848.083995,"POLYGON ((174.83900 -36.93816, 174.83932 -36.9...",2023-08-22 17:19:40.293544,25
1887,320700,Ilam South,Ilam South,0.911775,0.911775,5496.516868,"POLYGON ((172.57345 -43.51693, 172.57515 -43.5...",2023-08-22 17:19:40.298151,0
484,333100,Banks Peninsula South,Banks Peninsula South,371.610981,377.862913,181238.836589,"POLYGON ((172.66229 -43.68773, 172.67628 -43.6...",2023-08-22 17:19:40.313168,0
1110,173100,Eureka-Tauwhare,Eureka-Tauwhare,135.569365,135.569365,65989.866069,"POLYGON ((175.43812 -37.69567, 175.43838 -37.6...",2023-08-22 17:19:40.571617,18
1032,162000,Takanini West,Takanini West,0.686327,0.686327,4577.055752,"POLYGON ((174.92526 -37.04240, 174.92529 -37.0...",2023-08-22 17:19:40.921158,25
1659,249000,Broadmeadows,Broadmeadows,0.642137,0.642137,3753.041367,"POLYGON ((174.79257 -41.22976, 174.79309 -41.2...",2023-08-22 17:19:40.930975,0
956,158100,Ormiston South,Ormiston South,1.703467,1.703467,5866.134260,"POLYGON ((174.92198 -36.96751, 174.92271 -36.9...",2023-08-22 17:19:40.980087,3


In [61]:
pd.Series([r["properties"]["external_ids"].keys() for r in results.values()]).value_counts()

(foursquare)               66
(safegraph, foursquare)     5
dtype: int64

In [53]:
geojson = {"type": "FeatureCollection", "features": list(results.values())}
with open("data.geojson", "w") as f:
    json.dump(geojson, f)